In [40]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

In [20]:
dataSet = pd.read_csv('crawler/top20.csv')
dataSet = dataSet.drop(columns='Unnamed: 0', axis=1)

In [22]:
name_popId = dataSet[['name', 'popId']]

In [27]:
name_popId = name_popId.drop_duplicates().set_index('popId')

In [28]:
name_popId

,name
popId,
0,비타민 워터리 선 젤 SPF50+ PA++++
1,퓨어 미네랄 선크림 SPF50+ PA++++
2,썬 프로텍트 워터젤 SPF50 PA+++
3,산소수 라이트 선 젤 SPF50+ PA++++
4,유브이 퍼펙션 에브리데이썬 SPF50+ PA++++
5,수분 촉촉 데일리 선 로션 SPF30 PA++
6,UV 퍼펙트 에어쿨 선 젤 SPF50+ PA++++
7,유비데아 XL 멜트-인 내츄럴 크림 SPF50 PA++++
8,유브이 더블 컷 산뜻 선 젤 SPF50+ PA++++


In [30]:
dataSet = dataSet.drop('name', axis=1)

In [32]:
dataSet['rate'] = dataSet['rate']*20

In [35]:
dataSet['rate'] = dataSet['rate']/2

In [43]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words=['\r','\n'])
tfidf_matrix = tf.fit_transform(dataSet['review'])
print(tfidf_matrix)

  (0, 29466)	0.08424402094682544
  (0, 17294)	0.16848804189365088
  (0, 23386)	0.17773768418708163
  (0, 21965)	0.05929280503244968
  (0, 21895)	0.08636282673896047
  (0, 36202)	0.08886884209354082
  (0, 17201)	0.10146328612810616
  (0, 35520)	0.10146328612810616
  (0, 32462)	0.06914356155767974
  (0, 32207)	0.06620797904018029
  (0, 17212)	0.09589015586016997
  (0, 3409)	0.09193595700689668
  (0, 15554)	0.06424105358318563
  (0, 5575)	0.08424402094682544
  (0, 20670)	0.09589015586016997
  (0, 32616)	0.07376838270439512
  (0, 32098)	0.056680649918970795
  (0, 27827)	0.08636282673896047
  (0, 19922)	0.10146328612810616
  (0, 10931)	0.08886884209354082
  (0, 28931)	0.07471669182561595
  (0, 19861)	0.06135843849337616
  (0, 40665)	0.07683549761775098
  (0, 2457)	0.10146328612810616
  (0, 1975)	0.10146328612810616
  :	:
  (1180, 13522)	0.15176984245528174
  (1180, 31587)	0.11466555175011818
  (1180, 35961)	0.24162379916604934
  (1180, 34850)	0.08964814479309931
  (1180, 1908)	0.14131704110

In [ ]:
linear_kernel()